In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
import powerlaw
from statsmodels.stats import diagnostic
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

In [ ]:
from read_functions import *
from pickle_functions import *

In [ ]:
beer_advocate_path = '../datas/BeerAdvocate/'
matched_beer_data_path = '../datas/matched_beer_data/'
rate_beer_path = '../datas/RateBeer/'

advocate_beers_path = beer_advocate_path + 'beers.csv'
advovate_breweries_path = beer_advocate_path + 'breweries.csv'
advocate_ratings_path = beer_advocate_path + 'ratings.txt'
advocate_reviews_path = beer_advocate_path + 'reviews.txt'
advocate_users_path = beer_advocate_path + 'users.csv'

matched_beer_beers = matched_beer_data_path + 'beers.csv'
matched_beer_breweries = matched_beer_data_path + 'breweries.csv'
matched_beer_ratings_ba = matched_beer_data_path + 'ratings_ba.txt'
matched_beer_ratings_rb = matched_beer_data_path + 'ratings_rb.txt'
matched_beer_ratings = matched_beer_data_path + 'ratings.csv'
matched_beer_users_approx = matched_beer_data_path + 'users_approx.csv'
matched_beer_users = matched_beer_data_path + 'users.csv'

rate_beer_beers = rate_beer_path + 'beers.csv'
rate_beer_breweries = rate_beer_path + 'breweries.csv'
rate_beer_users = rate_beer_path + 'users.csv'
rate_beer_ratings = rate_beer_path + 'ratings.txt'
rate_beer_reviews = rate_beer_path + 'reviews.txt'

contries_path = '../datas/countries/countries.csv'


In [ ]:
beer_advocate_path = '../datas/BeerAdvocate/pickle/'
matched_beer_data_path = '../datas/matched_beer_data/pickle/'
rate_beer_path = '../datas/RateBeer/pickle/'

advocate_beers_path_pickle = beer_advocate_path + 'beers.csv'
advovate_breweries_path_pickle = beer_advocate_path + 'breweries.csv'
advocate_ratings_path_pickle = beer_advocate_path + 'ratings.txt'
advocate_reviews_path_pickle = beer_advocate_path + 'reviews.txt'
advocate_users_path_pickle = beer_advocate_path + 'users.csv'

matched_beer_beers_pickle = matched_beer_data_path + 'beers.csv'
matched_beer_breweries_pickle = matched_beer_data_path + 'breweries.csv'
matched_beer_ratings_ba_pickle = matched_beer_data_path + 'ratings_ba.txt'
matched_beer_ratings_rb_pickle = matched_beer_data_path + 'ratings_rb.txt'
matched_beer_ratings_pickle = matched_beer_data_path + 'ratings.csv'
matched_beer_users_approx_pickle = matched_beer_data_path + 'users_approx.csv'
matched_beer_users_pickle = matched_beer_data_path + 'users.csv'

rate_beer_beers_pickle = rate_beer_path + 'beers.csv'
rate_beer_breweries_pickle = rate_beer_path + 'breweries.csv'
rate_beer_users_pickle = rate_beer_path + 'users.csv'
rate_beer_ratings_pickle = rate_beer_path + 'ratings.txt'
rate_beer_reviews_pickle = rate_beer_path + 'reviews.txt'

In [ ]:
#Define list of datas for each website
datas_matched_beer_names = ['df_matched_beer_beers','df_matched_beer_breweries','df_matched_beer_ratings_ba','df_matched_beer_ratings_rb','df_matched_beer_ratings','df_matched_beer_users_approx','df_matched_beer_users']
datas_advocate_names = ['df_advocate_beers','df_advocate_breweries','df_advocate_ratings','df_advocate_reviews','df_advocate_users']
datas_rate_beer_names = ['df_rate_beer_beers','df_rate_beer_breweries','df_rate_beer_users','df_rate_beer_ratings','df_rate_beer_ratings']

### import datas 

### Import datas stored as pickle

In [ ]:
df_matched_beer_beers,df_matched_beer_breweries,df_matched_beer_ratings_ba,df_matched_beer_ratings_rb,df_matched_beer_ratings,df_matched_beer_users_approx,df_matched_beer_users = load_datas('matched_beer_data', datas_matched_beer_names)
df_rate_beer_beers,df_rate_beer_breweries,df_rate_beer_users = load_datas('RateBeer', datas_rate_beer_names)
df_advocate_beers,df_advocate_breweries,df_advocate_ratings,df_advocate_reviews,df_advocate_users = load_datas('BeerAdvocate', datas_advocate_names)

## Store dataFrame into picke format

In [ ]:
datas_rate_beer = [df_rate_beer_beers,df_rate_beer_breweries,df_rate_beer_users,df_rate_beer_ratings,df_rate_beer_reviews]
datas_matched_beer = [df_matched_beer_beers,df_matched_beer_breweries,df_matched_beer_ratings_ba,df_matched_beer_ratings_rb,df_matched_beer_ratings,df_matched_beer_users_approx,df_matched_beer_users]
datas_advocate_beer = [df_advocate_beers,df_advocate_breweries,df_advocate_ratings,df_advocate_reviews,df_advocate_users]

In [ ]:
#Saving datas
save_datas('RateBeer', datas_rate_beer,datas_rate_beer_names)
save_datas('matched_beer_data', datas_matched_beer,datas_matched_beer_names)

# Initial data analysis

## Analysis of the distribution
 1. Beer Advocate

In [ ]:
Advocate_users_per_country = deepcopy(pd.DataFrame(df_advocate_users.groupby('location')['user_id'].count()))
Advocate_users_per_country.columns = ['nb_users']
Advocate_users_per_country.sort_values(ascending=False, by='nb_users', inplace=True)
Advocate_users_per_country[Advocate_users_per_country.index.str.startswith('United States')].agg('sum')

Advocate_users_per_country.loc['United States', 'nb_users'] = Advocate_users_per_country[
    Advocate_users_per_country.index.str.startswith('United')
]['nb_users'].sum()
Advocate_users_per_country.drop(labels=Advocate_users_per_country[Advocate_users_per_country.index.str.startswith('United States,')].index, inplace=True)
Advocate_users_per_country.sort_values(ascending=False, by='nb_users', inplace=True)    

In [ ]:
sns.barplot(x=Advocate_users_per_country.index[:10], y=Advocate_users_per_country['nb_users'][:10])
plt.title('Number of users per country for BeerAdvocate')
plt.ylabel('Number of users')
plt.semilogy()
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.hist(data=df_advocate_users, x='nbr_ratings', bins=100000, cumulative=-1, histtype='step')
plt.title('Cumulative histogram of the number of ratings per user for BeerAdvocate')
plt.xlabel('Number of users')
plt.ylabel('Number of ratings')
plt.loglog()
plt.show()

In [ ]:
# Define the power law function
def power_law(x, alpha):
    return x**(-alpha)

In [ ]:
fit = powerlaw.Fit(df_advocate_users['nbr_ratings'].values)

print('alpha =', fit.alpha)

# R: loglikelihood ratio of the two distributions if positive the first distribution is better
R, p = fit.distribution_compare('power_law', 'exponential', normalized_ratio=True)
print('R =', R)
print('p =', p)

powerlaw.plot_pdf(df_advocate_users['nbr_ratings'].values, color='b')
fit.power_law.plot_pdf(color='r', linestyle='--')

plt.title('Power law distribution of the number of ratings per user for Beer Advocate')
plt.xlabel('Number of ratings')
plt.ylabel('Probability density')
plt.show()

2. Rate Beer

In [ ]:
rate_beer_users_per_country = deepcopy(pd.DataFrame(df_rate_beer_users.groupby('location')['user_id'].count()))
rate_beer_users_per_country.columns = ['nb_users']
rate_beer_users_per_country.sort_values(ascending=False, by='nb_users', inplace=True)

rate_beer_users_per_country.loc['United States', 'nb_users'] = rate_beer_users_per_country[
    rate_beer_users_per_country.index.str.startswith('United')
]['nb_users'].sum()
rate_beer_users_per_country.drop(labels=rate_beer_users_per_country[rate_beer_users_per_country.index.str.startswith('United States,')].index, inplace=True)
rate_beer_users_per_country.sort_values(ascending=False, by='nb_users', inplace=True)   

In [ ]:
sns.barplot(x=rate_beer_users_per_country.index[:10], y=rate_beer_users_per_country['nb_users'][:10])
plt.title('Number of users per country for Matched Beer')
plt.ylabel('Number of users')
plt.semilogy()
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.hist(data=df_rate_beer_users, x='nbr_ratings', bins=100000, cumulative=-1, histtype='step')
plt.title('Cumulative histogram of the number of ratings per user for Rate Beer')
plt.xlabel('Number of users')
plt.ylabel('Number of ratings')
plt.loglog()
plt.show()

In [ ]:
fit = powerlaw.Fit(df_rate_beer_users['nbr_ratings'].values)

print('alpha =', fit.alpha)

# R: loglikelihood ratio of the two distributions if positive the first distribution is better
R, p = fit.distribution_compare('power_law', 'exponential', normalized_ratio=True)
print('R =', R)
print('p =', p)

powerlaw.plot_pdf(df_rate_beer_users['nbr_ratings'].values, color='b')
fit.power_law.plot_pdf(color='r', linestyle='--')

plt.title('Power law distribution of the number of ratings per user for RateBeer')
plt.xlabel('Number of ratings')
plt.ylabel('Probability density')
plt.show()